In [4]:
#Code to get the word from terminal, obviously not used in this Notebook
#import sys
#word=sys.argv[0]

In [84]:
import math
#from scipy.stats import binom

def read_words(filename):
    f=open(filename)
    counters = {} #initialize empty dictionary
    tot=0
    for line in f: #iterate over the lines of the file
        for word in line.lower().split(): #convert to lower case and split lines
            if word.isalpha(): #check if the split word contains something other than letters
                if word in counters:
                    counters[word]+=1
                else:
                    counters[word]=1
                tot+=1
    f.close()
    priors={}
    for word in counters:
        priors[word]=counters[word]/tot #compute the relative frequency of each word P(C)
    return priors
            
DICT=read_words("big.txt")

def binomial(n,k,p):
    den=math.factorial(k)*math.factorial(n-k) 
    b_coeff = math.factorial(n)//den #binomial coefficient
    return b_coeff*(p**k)*(1.-p)**(n-k) #return binomial distribution value

def edit1(word): #function to tell the reachable words with 1 error given an input word
    n=len(word)
    letters = "abcdefghijklmnopqrstuvwxyz"
    var=set() #set removes duplicates, so we use it in place of a list
    for i in range(n+1):
        #Insertion
        for l in letters:
            nword=word[:i]+l+word[i:]
            var.add(nword)
            #Substitution of a letter
            if i<n:
                nword=word[:i]+l+word[i:]
                var.add(nword)
        #Deletion
        if i<n:
            nword=word[:i]+word[(i+1):]
            var.add(nword)
        #Consecutive Letters Flip
        if i+1<n:
            nword=word[:i]+word[i+1]+word[i]+word[(i+1):]
            var.add(nword)
    return var

def nedit(word,n): #function to tell the reachable words with 1 error given an input word
    var={word}
    for _ in range(n): #_ is used to indicate an unused variable
        nvar=set()
        for v in var:
            nvar|=edit1(v)
        var=nvar
    return var

word="testt"
#print(len(nedit(word,3)))

def correct_word(word, maxerrors, p, top): #top stand for the number of candidates
    candidates=[]
    for e in range(maxerrors+1):
        var=nedit(word,e)
        for c in var:
            if c in DICT:
                p_c=DICT[c]
                p_wc=binomial(len(c),e, p)
                score = p_c*p_wc
                candidates.append((score,c))
    candidates.sort(reverse=True)
    return candidates

candidates = correct_word(word,2,0.01,5)
print(candidates)
#print(DICTIONARY)

[(1.482973548323708e-06, 'test'), (7.414867741618542e-09, 'tests'), (3.890787218480148e-09, 'est')]


In [85]:
import sys
import math


# A simple spell corrector.  Inspired by the one found on Peter
# Norvig's page (https://norvig.com/spell-correct.html).
#
# It finds the corrcetion c maximizing the conditional probability
# P(c|w) that c is the "right" word given that the user entered w.
#
# By Bayes law:
#
# P(c|w) = P(w|c) P(c) / P(w)
#
# About these probabilities:
#
# - P(w|c) is modeled as a function of edits needed to tranform w into
#   c.  The number of edits is supposed to follow a binomial
#   distribution.
#
# - P(c) is the "language model", telling us how much c is likely to
#   occur.  This is estimated from data.
#
# - P(w) is the same for all c and can be ignored.
#
# The file big.txt is used to estimate P(c).  It can be downloaded
# from Norvig's page https://norvig.com/big.txt Any other large file
# of text could be used.


def read_words(filename):
    """Read the content of a file.

    Return a dictionary mapping (lowercase) words to probabilities.

    """
    f = open(filename)
    counters = {}
    total = 0
    for line in f:
        for word in line.lower().split():
            # exclude strings including non-alphabetic characters
            if word.isalpha():
                if word in counters:
                    counters[word] += 1
                else:
                    counters[word] = 1
                total += 1
    f.close()
    priors = {}
    for word in counters:
        priors[word] = counters[word] / total
    return priors


def binomial_coefficient(n, k):
    """Binomial coefficient 'n choose k'."""
    denominator = math.factorial(k) * math.factorial(n - k)
    return math.factorial(n) // denominator


def error_probability(e, n, q):
    """Probability of e errors on a word of length n."""
    binc = binomial_coefficient(n, e)
    return binc * (q ** e) * ((1 - q) ** (n - e))


def edit1(word):
    """Set of single editings starting from the word s."""
    n = len(word)
    letters = "abcdefghijklmnopqrstuvwxyz"
    variations = set()
    for i in range(n + 1):
        for l in letters:
            newword = word[:i] + l + word[i:]
            variations.add(newword)  # Insertion
            if i < n:
                newword = word[:i] + l + word[(i + 1):]
                variations.add(newword)  # Substitution
        if i < n:
            newword = word[:i] + word[(i + 1):]
            variations.add(newword)  # Deletion
        if i + 1 < n:
            newword = (word[:i] + word[i + 1] +
                       word[i] + word[(i + 1):])
            variations.add(newword)  # Inversion
    return variations


def edit_k(word, k):
    """Set of editings obtained with k operations from the given word."""
    variations = {word}
    for _ in range(k):
        newvars = set()
        for v in variations:
            newvars |= edit1(v)
        variations = newvars
    return variations


def correct_word(word, maxerrors, q, top):
    """Correct the word.

    Return a list of possible corrections starting from the most
    likely.
    """
    candidates = []
    for e in range(maxerrors + 1):
        variations = edit_k(word, e)
        for c in variations:
            if c in DICTIONARY:
                p_c = DICTIONARY[c]
                p_wc = error_probability(e, len(c), q)
                score = p_wc * p_c
                candidates.append((score, c))
    candidates.sort(reverse=True)
    return candidates[:top]


# Load the dictionary.
DICTIONARY = read_words("big.txt")

# Correct the word and print the corrections.
word = "tesst"
candidates = correct_word(word, 2, 0.01, 5)
for score, c in candidates:
    print(c)


test
less
least
best
west
